In [ ]:
import numpy as np
import pandas as pd

In [115]:
# note: after experimenting I realised np.diag() only creates a 
# strided view of the original data, not a regular 2D array, so 
# to make operations on it I need to add a float to it (here 0.2), 
# or just '+ 0.'
p = 3

def generateData(n):
    n1 = n2 = n//2
    cov_1 = np.diag(np.repeat(1, p)) + 0.2
    cov_2 = np.diag(np.repeat(1, p)) + 0.2
    cov_2[1, 2] = cov_2[2, 1] = cov_2[1, 2] + 0.5
    x_class1 = np.random.multivariate_normal(
        mean=np.repeat(3, p),
        cov=cov_1,
        size=n1
    )
    x_class2 = np.random.multivariate_normal(
        mean=np.repeat(2, p),
        cov=cov_2,
        size=n2
    )
    y = np.repeat((1, 2), (n1, n2))
    data_set = pd.concat([pd.DataFrame(x_class1), pd.DataFrame(x_class2)])
    data_set.columns = [f'x_{i}' for i in range(1, p+1)]
    data_set['y'] = y

    return data_set

Since the covariance matrices are the same for both classes, it follows the assumption that the validity (or let's say the performance) of LDA depends upon, so the bias is low. In this case, LDA might be the best performing classifier, while QDA would have have a higher variance would be too flexible for our data set here (given that the Bayes decision boundary is linear).

In [135]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis


def predictLDA(train_set, test_set):

    LDA = LinearDiscriminantAnalysis()
    LDA.fit(
        X=train_set[[f'x_{i}' for i in range(1, p+1)]], 
        y=train_set['y']
    )
    # let's first take a quick look at what's inside our LDA:
    # print("LDA Classes:", LDA.classes_)
    # print("LDA Means:", LDA.means_)
    # print("LDA Covariance matrix:", LDA.covariance_)

    y_pred = LDA.predict(X=test_set[[f'x_{i}' for i in range(1, p+1)]])

    return y_pred


def predictQDA(train_set, test_set):
    
    QDA = QuadraticDiscriminantAnalysis()
    QDA.fit(
        X=train_set[[f'x_{i}' for i in range(1, p+1)]], 
        y=train_set['y']
    )
    # let's first take a quick look at what's inside our QDA:
    # print("QDA Classes:", QDA.classes_)
    # print("QDA Means:", QDA.means_)
    # print("QDA Covariance matrix:", QDA.covariance_)

    y_pred = QDA.predict(X=test_set[[f'x_{i}' for i in range(1, p+1)]])

    return y_pred

First let's generate two small data sets and take a look at what our discriminant functions do:

In [117]:
train = generateData(20)
test = generateData(20)

In [118]:
predictLDA(train, test)

LDA Classes: [1 2]
LDA Means: [[3.22560067 2.39104093 3.38032654]
 [2.03234439 1.7851859  1.72716202]]
LDA Covariance matrix: [[ 1.13749041 -0.21471272 -0.20747685]
 [-0.21471272  0.70179146 -0.1659123 ]
 [-0.20747685 -0.1659123   0.94067546]]


c:\Users\Gaspard\miniconda3\envs\venv_stats\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([1, 1, 1, 2, 1, 1, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [ ]:
predictQDA(train, test)

QDA Classes: [1 2]
QDA Means: [[3.22560067 2.39104093 3.38032654]
 [2.03234439 1.7851859  1.72716202]]
QDA Covariance matrix: [array([[ 0.94555935, -0.23851467,  0.08941347],
       [-0.23851467,  0.4644733 , -0.49326297],
       [ 0.08941347, -0.49326297,  1.17158881]]), array([[ 1.58219712, -0.23862471, -0.55047315],
       [-0.23862471,  1.09506328,  0.12456897],
       [-0.55047315,  0.12456897,  0.91880109]])]


c:\Users\Gaspard\miniconda3\envs\venv_stats\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([1, 1, 1, 2, 2, 1, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

As we can see, the QDA function returned two covariances matrices. Also notice how the fifth elements of the ```y_pred``` results differ betwee the two methods, but the rest of the predicted labels are the same.

I have now removed the ```store_covariance=True``` parameter and commented out the print statements in the discriminant functions, to prepare for the loop we are going to be running next.

Below I manually coded a function that computes the confusion matrix and returns the accuracy, for the challenge:

In [133]:
def classifyPreds(row):
    
    if row.y == 2 and row.y_hat == 2:
        return 'TP'
    elif row.y == 2 and row.y_hat == 1:
        return 'FN'
    elif row.y == 1 and row.y_hat == 2:
        return 'FP'
    elif row.y == 1 and row.y_hat == 1:
        return 'TN'


def getAccuracy(df_labels):

    pos_negs = df_labels.apply(classifyPreds, axis=1)
    cf_matrix = pos_negs.value_counts()
    return (cf_matrix['TP'] + cf_matrix['TN']) / sum(cf_matrix)

And now the function that is going to run the experiment 100 times and return the average accuracy over these 100 repetitions for both methods:

In [131]:
from statistics import fmean


def discriminantAnalysis(n_train):
    
    classifier_acc = {'LDA': [], 'QDA': []}

    for _ in range(100):
        train = generateData(n_train)
        test = generateData(10_000)

        y_hat_LDA = predictLDA(train, test)
        y_hat_QDA = predictQDA(train, test)

        df_LDA = pd.DataFrame({'y': test['y'], 'y_hat': y_hat_LDA})
        df_QDA = pd.DataFrame({'y': test['y'], 'y_hat': y_hat_QDA})

        classifier_acc['LDA'].append(getAccuracy(df_LDA))
        classifier_acc['QDA'].append(getAccuracy(df_QDA))

    return pd.DataFrame({'LDA': [round(fmean(classifier_acc['LDA']), 4)], 'QDA': [round(fmean(classifier_acc['QDA']), 4)]})

In [136]:
df_train_50 = discriminantAnalysis(50)
df_train_10k = discriminantAnalysis(10_000)

df_results = pd.concat([df_train_50, df_train_10k])
df_results.index = pd.Index(['n_train=50', 'n_train=10^4'])
df_results

,LDA,QDA
n_train=50,0.7276,0.7264
n_train=10^4,0.7446,0.7577


In the table above, we see that for ```n_train = 50```, over 100 repetitions, LDA and QDA have virtually the same accuracy. I would say this result here is not exactly in line with my expectations because with as few as 50 training observations, I would have expected LDA to perform better because it has lower variance and also the distributions of our X predictors in both classes have the same covariance matrix. For ```n_train = 10,000``` though, we see that QDA barely outperforms LDA: I cannot say I am surprised at this result this time because with such a bigger training set, our models can "see" most of the variance in the data, and having a more flexible model is less of an issue in this case and it also has lower bias. Even with this training set size, QDA does not vastly outperform LDA, because once again the assumption of equal covariance matrices perfectly holds.